In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
dfGPS = pd.read_excel("../input_7.1.2018/GPS_Mapa_Krzysiek.xlsx")
# Ładowanie siatki predykcji
df = pd.read_csv("../input/gps_10_5.csv", header=None)

In [5]:
dfGPS.iloc[:,2:16].sample(5)

,Miejscowość,IDKSIP zdarzenia,Dzień,Numer drogi,Ulica,GPS x,GPS y,Zachowanie kierującego,Zachowanie pieszego,Inne przyczyny,Liczba wypadków,Liczba zabitych w zdarzeniu,Liczba rannych w zdarzeniu,Liczba kolizji
97942,OCHABY MAŁE,EWK-105491259F,2017-02-27,81,KATOWICKA,18*46'079,49*50'322,Niestosowanie się do sygnalizacji świetlnej,NaN,NaN,0,0,0,1
38131,BIERUŃ,EWK-104815256A,2016-09-06,W931,TURYSTYCZNA,19*06'514,50*04'004,Nieustąpienie pierwszeństwa przejazdu,NaN,NaN,0,0,0,1
84534,CZERWIONKA-LESZCZYNY,EWK-106609112A,2017-12-10,A1A,NaN,18*42'474,50*10'401,Inne przyczyny,NaN,NaN,0,0,0,1
27691,PIEKARY ŚLĄSKIE,EWK-105296690R,2016-12-20,I2,BYTOMSKA,18*56'371,50*22'105,Nieprawidłowe cofanie,NaN,NaN,0,0,0,1
110778,ŻYWIEC,EWK-106410127H,2017-09-18,W069,WESOŁA,19*10'526,49*41'353,Niedostosowanie prędkości do warunków ruchu,NaN,NaN,0,0,0,1


In [6]:
dfGPS['Miejscowość'] = dfGPS['Miejscowość'].map(lambda x: x.lower())
dfGPS['Miejscowość'].value_counts(normalize=True) 
vc =dfGPS['Miejscowość'].value_counts().reset_index()
vc
other_city = set(vc[vc.Miejscowość<10]['index'].unique())
dfGPS['Miejscowość_cat'] = dfGPS['Miejscowość'].map(lambda x: 'other' if x in other_city else x)
dfGPS['Miejscowość_cat'].value_counts()
dfGPS['Miejscowość_cat'] = pd.factorize( dfGPS['Miejscowość_cat'])[0]
dfGPS['Miejscowość_cat'] = dfGPS['Miejscowość_cat'].map(lambda x: x+1)

In [7]:
dfGPS['Numer_drogi_cat'] = dfGPS['Numer drogi'].fillna(-1)
dfGPS['Numer drogi'].value_counts(normalize=True) 
vc =dfGPS['Numer drogi'].value_counts().reset_index()
other_city = set(vc[vc['Numer drogi']<2]['index'].unique())
dfGPS['Numer_drogi_cat'] = dfGPS['Numer drogi'].map(lambda x: 'other' if x in other_city else x)
dfGPS['Numer_drogi_cat'].value_counts()
dfGPS['Numer_drogi_cat'] = pd.factorize( dfGPS['Numer_drogi_cat'])[0]
dfGPS['Numer_drogi_cat'] = dfGPS['Numer_drogi_cat'].map(lambda x: x+1)

dfGPS['MiejscowoscUlica'] = dfGPS['Miejscowość'] + dfGPS['Ulica']
dfGPS['MiejscowoscUlica'].fillna(-1, inplace=True)
dfGPS['MiejscowoscUlica'].value_counts(normalize=True) 
vc =dfGPS['MiejscowoscUlica'].value_counts().reset_index()
other_city = set(vc[vc['MiejscowoscUlica']==1]['index'].unique())
dfGPS['MiejscowoscUlica'] = dfGPS['MiejscowoscUlica'].map(lambda x: 'other' if x in other_city else x)
dfGPS['MiejscowoscUlica'] = pd.factorize( dfGPS['MiejscowoscUlica'])[0]
dfGPS['MiejscowoscUlica'] = dfGPS['MiejscowoscUlica'].map(lambda x: x+1)

In [8]:
dfGPS2016 = dfGPS[dfGPS['rok']==2016]
dfGPS2017 = dfGPS[dfGPS['rok']==2017]
dfGPS2018 = dfGPS[dfGPS['rok']==2018]

In [360]:
df[6] = df[0].map(lambda x:0) #zdarzenia
df[7] = df[0].map(lambda x:0) #wypadke
df[8] = df[0].map(lambda x:0) #zabity
df[9] = df[0].map(lambda x:0) #ranny
df[10] = df[0].map(lambda x:0) #miejscowość
df[11] = df[0].map(lambda x:0) #droga
df[12] = df[0].map(lambda x:0) #miejscowość ulica
df[13] = df[0].map(lambda x:0)
df2016a = df.copy()
df2017a = df.copy()
df2018a = df.copy()

In [10]:
dfGPS2016.loc[:,'GPS_X_sekundy'] = dfGPS2016.loc[:,'GPS_X_sekundy'].map(lambda x:x/100)
dfGPS2016.loc[:,'GPS_X_sekundy'] = dfGPS2016.loc[:,'GPS_X_sekundy'].astype('int8')
dfGPS2016.loc[:,'GPS_Y_sekundy'] = dfGPS2016.loc[:,'GPS_Y_sekundy'].map(lambda x:x/50)
dfGPS2016.loc[:,'GPS_Y_sekundy'] = dfGPS2016.loc[:,'GPS_Y_sekundy'].astype('int8')

/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [11]:
dfGPS2017.loc[:,'GPS_X_sekundy'] = dfGPS2017.loc[:,'GPS_X_sekundy'].map(lambda x:x/100)
dfGPS2017.loc[:,'GPS_X_sekundy'] = dfGPS2017.loc[:,'GPS_X_sekundy'].astype('int8')
dfGPS2017.loc[:,'GPS_Y_sekundy'] = dfGPS2017.loc[:,'GPS_Y_sekundy'].map(lambda x:x/50)
dfGPS2017.loc[:,'GPS_Y_sekundy'] = dfGPS2017.loc[:,'GPS_Y_sekundy'].astype('int8')

/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [12]:
dfGPS2018.loc[:,'GPS_X_sekundy'] = dfGPS2018.loc[:,'GPS_X_sekundy'].map(lambda x:x/100)
dfGPS2018.loc[:,'GPS_X_sekundy'] = dfGPS2018.loc[:,'GPS_X_sekundy'].astype('int8')
dfGPS2018.loc[:,'GPS_Y_sekundy'] = dfGPS2018.loc[:,'GPS_Y_sekundy'].map(lambda x:x/50)
dfGPS2018.loc[:,'GPS_Y_sekundy'] = dfGPS2018.loc[:,'GPS_Y_sekundy'].astype('int8')

/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [59]:
for i in  tqdm(range(dfGPS2016.shape[0])):
    
    a = df2016a.iloc[:,0]==dfGPS2016.iloc[i,20]
    b = df2016a.iloc[:,1]==dfGPS2016.iloc[i,21]
    c = df2016a.iloc[:,2]==dfGPS2016.iloc[i,22]
    d = df2016a.iloc[:,3]==dfGPS2016.iloc[i,23]
    e = df2016a.iloc[:,4]==dfGPS2016.iloc[i,24]
    f = df2016a.iloc[:,5]==dfGPS2016.iloc[i,25]
    x = df2016a[a & b & c & d & e & f].index
    if len (x) != 0:
        df2016a.iloc[x[0],6]+=1
        df2016a.iloc[x[0],7]+=dfGPS2016.iloc[i,12]
        df2016a.iloc[x[0],8]+=dfGPS2016.iloc[i,13]
        df2016a.iloc[x[0],9]+=dfGPS2016.iloc[i,14]
        df2016a.iloc[x[0],10]=dfGPS2016.iloc[i,26]
        df2016a.iloc[x[0],11]=dfGPS2016.iloc[i,27]
        df2016a.iloc[x[0],12]=dfGPS2016.iloc[i,28]

100%|██████████| 53260/53260 [19:20<00:00, 45.90it/s]


In [60]:
for i in  tqdm(range(dfGPS2017.shape[0])):
    
    a = df2017a.iloc[:,0]==dfGPS2017.iloc[i,20]
    b = df2017a.iloc[:,1]==dfGPS2017.iloc[i,21]
    c = df2017a.iloc[:,2]==dfGPS2017.iloc[i,22]
    d = df2017a.iloc[:,3]==dfGPS2017.iloc[i,23]
    e = df2017a.iloc[:,4]==dfGPS2017.iloc[i,24]
    f = df2017a.iloc[:,5]==dfGPS2017.iloc[i,25]
    x = df2017a[a & b & c & d & e & f].index
    if len (x) != 0:
        df2017a.iloc[x[0],6]+=1
        df2017a.iloc[x[0],7]+=dfGPS2017.iloc[i,12]
        df2017a.iloc[x[0],8]+=dfGPS2017.iloc[i,13]
        df2017a.iloc[x[0],9]+=dfGPS2017.iloc[i,14]
        df2017a.iloc[x[0],10]=dfGPS2017.iloc[i,26]
        df2017a.iloc[x[0],11]=dfGPS2017.iloc[i,27]
        df2017a.iloc[x[0],12]=dfGPS2017.iloc[i,28]

100%|██████████| 57668/57668 [20:14<00:00, 47.48it/s]


In [13]:
for i in  tqdm(range(dfGPS2018.shape[0])):
    
    a = df2018a.iloc[:,0]==dfGPS2018.iloc[i,20]
    b = df2018a.iloc[:,1]==dfGPS2018.iloc[i,21]
    c = df2018a.iloc[:,2]==dfGPS2018.iloc[i,22]
    d = df2018a.iloc[:,3]==dfGPS2018.iloc[i,23]
    e = df2018a.iloc[:,4]==dfGPS2018.iloc[i,24]
    f = df2018a.iloc[:,5]==dfGPS2018.iloc[i,25]
    x = df2018a[a & b & c & d & e & f].index
    if len (x) != 0:
        df2018a.iloc[x[0],6]+=1
        df2018a.iloc[x[0],7]+=dfGPS2018.iloc[i,12]
        df2018a.iloc[x[0],8]+=dfGPS2018.iloc[i,13]
        df2018a.iloc[x[0],9]+=dfGPS2018.iloc[i,14]
        df2018a.iloc[x[0],10]=dfGPS2018.iloc[i,26]
        df2018a.iloc[x[0],11]=dfGPS2018.iloc[i,27]
        df2018a.iloc[x[0],12]=dfGPS2018.iloc[i,28]

100%|██████████| 57634/57634 [17:50<00:00, 53.85it/s]


In [66]:
df2016a.to_csv('../output/GPS2016c.csv', index=False, header=True)
df2017a.to_csv('../output/GPS2017c.csv', index=False, header=True)
df2018a.to_csv('../output/GPS2018c.csv', index=False, header=True)